In [5]:
from collections import defaultdict

def probability_X_gt_Y(n: int):
    if n <= 1:
        return 0.0, 1.0, 0.0

    probs = {(1, 0): 0.5, (0, 0): 0.5}
    for _ in range(2, n+1):
        new_probs = defaultdict(float)
        for (last, diff), prob in probs.items():
            if last == 1:
                new_probs[(1, diff-1)] += prob * 0.5
                new_probs[(0, diff+1)] += prob * 0.5
            else:
                new_probs[(1, diff)] += prob * 0.5
                new_probs[(0, diff)] += prob * 0.5

        probs = new_probs

    total_or = 0
    total_oo = 0

    for (last, diff), prob in probs.items():
        if diff > 0:
            total_or += prob
        elif diff < 0:
            total_oo += prob
    return total_or, 1-total_or-total_oo, total_oo

probability_X_gt_Y(20)
                

(0.4677152633666992, 0.12754154205322266, 0.4047431945800781)

In [82]:
import json

with open("01", "rb") as f:
    a = json.load(f)

serpset = a['serpset']
new_docs = a['new_documents']
serpset

[{'query': 'okna',
  'results': [{'position': 0,
    'url': 'okna-msk.ru',
    'relevance': 0.95,
    'cost': 77},
   {'position': 1, 'url': 'okna-pvh.ru', 'relevance': 0.95, 'cost': 70},
   {'position': 2, 'url': 'ne-okna.ru', 'relevance': 0.3, 'cost': 100},
   {'position': 3, 'url': 'best-okna.ru', 'relevance': 0.1, 'cost': 0}]},
 {'query': 'lego',
  'results': [{'position': 0, 'url': 'lego.ru', 'relevance': 0.95, 'cost': 15},
   {'position': 1, 'url': 'lego-mir.ru', 'relevance': 0.7, 'cost': 30},
   {'position': 2, 'url': 'disney.ru', 'relevance': 0.3, 'cost': 100}]}]

In [83]:
serpset_query_map = {s['query']: s['results'] for s in serpset}
serpset_query_map

{'okna': [{'position': 0, 'url': 'okna-msk.ru', 'relevance': 0.95, 'cost': 77},
  {'position': 1, 'url': 'okna-pvh.ru', 'relevance': 0.95, 'cost': 70},
  {'position': 2, 'url': 'ne-okna.ru', 'relevance': 0.3, 'cost': 100},
  {'position': 3, 'url': 'best-okna.ru', 'relevance': 0.1, 'cost': 0}],
 'lego': [{'position': 0, 'url': 'lego.ru', 'relevance': 0.95, 'cost': 15},
  {'position': 1, 'url': 'lego-mir.ru', 'relevance': 0.7, 'cost': 30},
  {'position': 2, 'url': 'disney.ru', 'relevance': 0.3, 'cost': 100}]}

In [84]:
def update_pos(positions, l, query):
    for i, pos in enumerate(positions):
        if i == l:
            break
        pos['position'] = i
        pos['query'] = query
        positions[i] = pos
    positions = positions[:l]
    return positions    

In [ ]:
for doc in new_docs:
    if doc['query'] not in serpset_query_map:
        continue

    curr_positions = serpset_query_map[doc['query']]
    if doc['relevance'] < curr_positions[-1]['relevance']:
        continue

    l = len(curr_positions)
    # print(l)
    for i, pos in enumerate(curr_positions[:]):
        if doc['relevance'] >= pos['relevance'] and doc['cost'] >= pos['cost']:
            curr_positions.insert(i, doc)
            break
    # print(l)

    # serpset_query_map[doc['query']] = curr_positions
    serpset_query_map[doc['query']] = update_pos(curr_positions, l, doc['query'])

4
4
3
3


In [87]:
serpset_query_map['lego']

[{'position': 0,
  'url': 'lego.ru',
  'relevance': 0.95,
  'cost': 15,
  'query': 'lego'},
 {'position': 1,
  'url': 'lego-mir.ru',
  'relevance': 0.7,
  'cost': 30,
  'query': 'lego'},
 {'position': 2,
  'url': 'disney.ru',
  'relevance': 0.3,
  'cost': 100,
  'query': 'lego'}]

In [75]:
new_docs

[{'query': 'weather', 'url': 'yandex.ru/pogoda', 'relevance': 1, 'cost': 0},
 {'query': 'okna',
  'url': 'yandex.ru/okna',
  'relevance': 1,
  'cost': 100,
  'position': 0},
 {'query': 'lego', 'url': 'yandex.ru/lego', 'relevance': 0.4, 'cost': 10}]

In [4]:
serpset_okna = serpset[0]
serpset_okna

{'query': 'okna',
 'results': [{'position': 0,
   'url': 'okna-msk.ru',
   'relevance': 0.95,
   'cost': 77},
  {'position': 1, 'url': 'okna-pvh.ru', 'relevance': 0.95, 'cost': 70},
  {'position': 2, 'url': 'ne-okna.ru', 'relevance': 0.3, 'cost': 100},
  {'position': 3, 'url': 'best-okna.ru', 'relevance': 0.1, 'cost': 0}]}

In [97]:
def current_relevance_cost(positions):
    relevance = 0
    cost = 0
    for i, pos in enumerate(positions):
        rel = pos['relevance']
        c = pos['cost']
        relevance += rel / (i+1)
        cost += c / (i+1)**0.5
    return relevance, cost


ans = 0
for k in serpset_query_map:
    # print(k)
    ans += current_relevance_cost(serpset_query_map[k])[1]
# current_relevance_cost(serpset_query_map['okna'])[1] + current_relevance_cost(serpset_query_map['lego'])[1]
ans

338.809971349197

In [ ]:
import json
import math
from typing import List, Dict, Any

def calculate_rel_dcg(results: List[Dict]) -> float:
    """Вычисляет RelDCG для списка результатов"""
    return sum(doc['relevance'] / (i + 1) for i, doc in enumerate(results))

def calculate_revenue(results: List[Dict]) -> float:
    """Вычисляет Revenue для списка результатов"""
    return sum(doc['cost'] / math.sqrt(i + 1) for i, doc in enumerate(results))

def find_best_insertion_position(original_results: List[Dict], new_doc: Dict) -> tuple:
    """Находит лучшую позицию для вставки нового документа"""
    original_rel_dcg = calculate_rel_dcg(original_results)
    original_revenue = calculate_revenue(original_results)
    
    best_position = -1
    best_revenue_gain = -float('inf')
    best_results = None
    
    # Пробуем вставить на каждую возможную позицию
    for pos in range(len(original_results)):
        # Создаем новую выдачу: вставляем новый документ, удаляем последний
        new_results = original_results[:pos] + [new_doc] + original_results[pos:-1]
        
        # Проверяем, что качество не ухудшилось
        new_rel_dcg = calculate_rel_dcg(new_results)
        if new_rel_dcg < original_rel_dcg:
            continue
        
        # Вычисляем прирост выручки
        new_revenue = calculate_revenue(new_results)
        revenue_gain = new_revenue - original_revenue
        
        if revenue_gain > best_revenue_gain:
            best_revenue_gain = revenue_gain
            best_position = pos
            best_results = new_results
    
    return best_position, best_results, best_revenue_gain

def optimize_serp(data: Dict[str, Any]) -> float:
    """Оптимизирует поисковую выдачу и возвращает максимальную суммарную выручку"""
    serpset = data['serpset']
    new_documents = data['new_documents']
    
    # Группируем новые документы по запросам
    new_docs_by_query = {}
    for doc in new_documents:
        query = doc['query']
        if query not in new_docs_by_query:
            new_docs_by_query[query] = []
        new_docs_by_query[query].append(doc)
    
    total_revenue = 0
    
    # Обрабатываем каждый запрос
    for serp in serpset:
        query = serp['query']
        results = serp['results'].copy()
        
        # Если есть новые документы для этого запроса
        if query in new_docs_by_query:
            # Сортируем новые документы по убыванию потенциальной выгоды
            candidates = new_docs_by_query[query].copy()
            candidates.sort(key=lambda x: x['relevance'] * x['cost'], reverse=True)
            
            # Пробуем вставить каждый документ
            for new_doc in candidates:
                best_pos, best_results, revenue_gain = find_best_insertion_position(results, new_doc)
                if best_pos != -1 and revenue_gain > 0:
                    results = best_results
        
        # Добавляем выручку от этой выдачи
        total_revenue += calculate_revenue(results)
    
    return total_revenue

with open("input.txt", "rb") as f:
    data = json.load(f)
result = optimize_serp(data)
print(f"{result:.2f}")

# Пример использования
example_data = {
    "serpset": [
        {
            "query": "okna",
            "results": [
                {"position": 0, "url": "okna-msk.ru", "relevance": 0.95, "cost": 77},
                {"position": 1, "url": "okna-pvh.ru", "relevance": 0.95, "cost": 70},
                {"position": 2, "url": "ne-okna.ru", "relevance": 0.3, "cost": 100},
                {"position": 3, "url": "best-okna.ru", "relevance": 0.1, "cost": 0}
            ]
        },
        {
            "query": "lego",
            "results": [
                {"position": 0, "url": "lego.ru", "relevance": 0.95, "cost": 15},
                {"position": 1, "url": "lego-mir.ru", "relevance": 0.7, "cost": 30},
                {"position": 2, "url": "disney.ru", "relevance": 0.3, "cost": 100}
            ]
        }
    ],
    "new_documents": [
        {"query": "weather", "url": "yandex.ru/pogoda", "relevance": 1, "cost": 0},
        {"query": "okna", "url": "yandex.ru/okna", "relevance": 1, "cost": 100},
        {"query": "lego", "url": "yandex.ru/lego", "relevance": 0.4, "cost": 10}
    ]
}
# print(f"Максимальная суммарная выручка: {result:.2f}")

# # Проверим вычисления вручную
# print("\nПроверка вычислений:")

# # Исходная выдача для "okna"
# okna_original = [
#     {"url": "okna-msk.ru", "relevance": 0.95, "cost": 77},
#     {"url": "okna-pvh.ru", "relevance": 0.95, "cost": 70}, 
#     {"url": "ne-okna.ru", "relevance": 0.3, "cost": 100},
#     {"url": "best-okna.ru", "relevance": 0.1, "cost": 0}
# ]

# print("Okna original:")
# print(f"RelDCG = {calculate_rel_dcg(okna_original):.4f}")
# print(f"Revenue = {calculate_revenue(okna_original):.2f}")

# # Проверим вставку yandex.ru/okna на разные позиции
# new_okna_doc = {"url": "yandex.ru/okna", "relevance": 1, "cost": 100}

# for pos in range(4):
#     new_results = okna_original[:pos] + [new_okna_doc] + okna_original[pos:-1]
#     rel_dcg = calculate_rel_dcg(new_results)
#     revenue = calculate_revenue(new_results)
#     print(f"Позиция {pos}: RelDCG={rel_dcg:.4f}, Revenue={revenue:.2f}, Gain={revenue - calculate_revenue(okna_original):.2f}")

# # Исходная выдача для "lego"
# lego_original = [
#     {"url": "lego.ru", "relevance": 0.95, "cost": 15},
#     {"url": "lego-mir.ru", "relevance": 0.7, "cost": 30},
#     {"url": "disney.ru", "relevance": 0.3, "cost": 100}
# ]

# print(f"\nLego original:")
# print(f"RelDCG = {calculate_rel_dcg(lego_original):.4f}")
# print(f"Revenue = {calculate_revenue(lego_original):.2f}")

# # Проверим вставку yandex.ru/lego
# new_lego_doc = {"url": "yandex.ru/lego", "relevance": 0.4, "cost": 10}

# for pos in range(3):
#     new_results = lego_original[:pos] + [new_lego_doc] + lego_original[pos:-1]
#     rel_dcg = calculate_rel_dcg(new_results)
#     revenue = calculate_revenue(new_results)
#     gain = revenue - calculate_revenue(lego_original)
#     quality_ok = rel_dcg >= calculate_rel_dcg(lego_original)
#     print(f"Позиция {pos}: RelDCG={rel_dcg:.4f}, Revenue={revenue:.2f}, Gain={gain:.2f}, Quality OK: {quality_ok}")

# print(f"\nОбщий исходный Revenue: {calculate_revenue(okna_original) + calculate_revenue(lego_original):.2f}")

# # Дополнительный анализ для проверки
# print("\nДетальный анализ:")

# def analyze_query(serp, new_docs_for_query):
#     """Анализирует оптимизацию для конкретного запроса"""
#     print(f"\nЗапрос: {serp['query']}")
#     original_results = serp['results']
    
#     print(f"Исходная RelDCG: {calculate_rel_dcg(original_results):.4f}")
#     print(f"Исходный Revenue: {calculate_revenue(original_results):.2f}")
    
#     if new_docs_for_query:
#         for new_doc in new_docs_for_query:
#             best_pos = find_best_insertion_position(original_results, new_doc)
#             if best_pos != -1:
#                 new_results = original_results[:best_pos] + [new_doc] + original_results[best_pos:-1]
#                 print(f"Вставка '{new_doc['url']}' на позицию {best_pos}")
#                 print(f"Новая RelDCG: {calculate_rel_dcg(new_results):.4f}")
#                 print(f"Новый Revenue: {calculate_revenue(new_results):.2f}")
#                 original_results = new_results
#             else:
#                 print(f"Документ '{new_doc['url']}' не улучшает качество")

# # Группируем для анализа
# new_docs_by_query = {}
# for doc in example_data['new_documents']:
#     query = doc['query']
#     if query not in new_docs_by_query:
#         new_docs_by_query[query] = []
#     new_docs_by_query[query].append(doc)

# for serp in example_data['serpset']:
#     query = serp['query']
#     analyze_query(serp, new_docs_by_query.get(query, []))

338.81
